![Sales Forcasting](https://triuminfo.com/application/views/images/sales_forecasting_banner.jpg)

## 📚 Libraries 

In [1]:
import pandas as pd 
import numpy as np 
import plotly.express as px 


In [2]:
!pip install fastparquet


df = pd.read_csv("datasets/train.csv")

df.to_parquet("train.parquet")


#### We needed to do so because train.csv was so big to be uploaded to github